In [1]:
import _pickle as cPickle
import docplex
from docplex.mp.model import Model

In [2]:
with open(r"../Data/test_optimize.pickle", "rb") as input_file:
    df = cPickle.load(input_file)

with open(r"../Data/team.pickle", "rb") as input_file:
    team = cPickle.load(input_file)

In [3]:
team_dict = dict(zip(team.code.to_list(),team.short_name.to_list()))

In [4]:
df

,id,web_name,team,element_type,now_cost,predict_points,actual_points
0,1,Cédric,1,2,42,1.452501,2
1,3,Xhaka,1,3,51,56.542370,62
2,4,Elneny,1,3,42,2.317113,4
3,5,Holding,1,2,42,1.452501,4
4,6,Partey,1,3,48,38.903732,42
...,...,...,...,...,...,...,...
662,625,Diego Costa,20,4,55,9.489326,7
663,629,Traoré,20,3,45,10.521433,12
664,657,Fraser,20,4,45,0.350952,0
665,658,Griffiths,20,3,45,0.350952,0


In [5]:
import scipy
import numpy as np
from scipy.optimize import minimize
import pulp

In [6]:
def select_team(expected_scores, prices, positions, clubs, total_budget=100, sub_factor=0.2):
    num_players = len(expected_scores)
    model = pulp.LpProblem("Constrained value maximisation", pulp.LpMaximize)
    decisions = [
        pulp.LpVariable("x{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    captain_decisions = [
        pulp.LpVariable("y{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    sub_decisions = [
        pulp.LpVariable("z{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ] 


    # objective function:
    model += sum((captain_decisions[i] + decisions[i] + sub_decisions[i]*sub_factor) * expected_scores[i]
                 for i in range(num_players)), "Objective"

    # cost constraint
    model += sum((decisions[i] + sub_decisions[i]) * prices[i] for i in range(num_players)) <= total_budget  # total cost

    # position constraints
    # 1 starting goalkeeper
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 1) == 1
    # 2 total goalkeepers
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 1) == 2

    # 3-5 starting defenders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 2) >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 2) <= 5
    # 5 total defenders
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 2) == 5

    # 3-5 starting midfielders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 3) >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 3) <= 5
    # 5 total midfielders
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 3) == 5

    # 1-3 starting attackers
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 4) >= 1
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 4) <= 3
    # 3 total attackers
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 4) == 3

    # club constraint
    for club_id in np.unique(clubs):
        model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if clubs[i] == club_id) <= 3  # max 3 players

    model += sum(decisions) == 11  # total team size
    model += sum(captain_decisions) == 1  # 1 captain
    
    for i in range(num_players):  
        model += (decisions[i] - captain_decisions[i]) >= 0  # captain must also be on team
        model += (decisions[i] + sub_decisions[i]) <= 1  # subs must not be on team

    model.solve()
    print("Total expected score = {}".format(model.objective.value()))

    return decisions, captain_decisions, sub_decisions

In [7]:
df.columns

Index(['id', 'web_name', 'team', 'element_type', 'now_cost', 'predict_points',
       'actual_points'],
      dtype='object')

In [8]:
expected_scores = df.predict_points
prices = df.now_cost / 10
position = df.element_type
team = df.team
names = df.web_name

decisions, captain_decisions, sub_decisions = select_team(expected_scores.values, prices.values, position.values, team.values)

/home/caleb/PythonEnvs/FPL/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/caleb/PythonEnvs/FPL/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/83ac30aba6cc46b68b9b5be6cb25eb2e-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/83ac30aba6cc46b68b9b5be6cb25eb2e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1373 COLUMNS
At line 16645 RHS
At line 18014 BOUNDS
At line 20016 ENDATA
Problem MODEL has 1368 rows, 2001 columns and 9268 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1075.42 - 0.00 seconds
Cgl0004I processed model has 1365 rows, 2001 columns (2001 integer (2001 of which binary)) and 8671 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.451613
Cbc0038I Pass   1: suminf.    0.33735 (2) obj. -1063.69 iterations 17
Cbc0038I Solution found of -988.023
Cbc0038I Before mini branch and bound, 1998 intege

In [9]:
for i in range(df.shape[0]):
    if decisions[i].value() != 0:
        print(f'{names[i]}: {expected_scores[i]} points at ${prices[i]}')
print('subs')
for i in range(df.shape[0]):
    if sub_decisions[i].value() != 0:
        print(f'{names[i]}: {expected_scores[i]} points at ${prices[i]}')
print('Capt')
#for i in range(df.shape[0]):
#    if captain_decisions[i].value != 0:
#        print(f'Captain: {names[i]} score: {captain_decisions[i].value}')

Ødegaard: 75.31554412841797 points at $6.4
Toney: 83.4347915649414 points at $7.4
Raya: 58.46512985229492 points at $4.6
Trossard: 87.79013061523438 points at $7.1
Tarkowski: 60.165733337402344 points at $4.4
Castagne: 59.329505920410156 points at $4.7
De Bruyne: 91.37598419189453 points at $12.6
Haaland: 116.27536010742188 points at $12.2
Trippier: 88.82398986816406 points at $5.9
Almirón: 90.53358459472656 points at $5.8
Kane: 99.83307647705078 points at $11.6
subs
Mee: 55.976802825927734 points at $4.6
Andreas: 59.94528579711914 points at $4.6
Ward: 53.411529541015625 points at $4.1
N.Williams: 48.06904602050781 points at $4.0
Capt
